In [ ]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

In [ ]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

##### Assigning key variables

In [ ]:
# define some variables to be used throughout the notebook
db1 = "familyman_derived_dev_v2" #database where Familyman data is stored
db2 = "familyman_dev_v2"
snapshot_date = "2022-05-23" #To update where necessary
db3 = "fcsq" #database where tables created as part of FCSQ processing are stored where required

In [ ]:
pydb.create_temp_table(
f""" 
SELECT 
  case_number,
  CAST(value AS Date) AS case_DOI
FROM 
  {db2}.case_fields AS f
 
WHERE
  field_model = 'FM2C_DI'
  AND mojap_snapshot_date = DATE'{snapshot_date}'

""",

"DOI")

In [ ]:
pydb.create_temp_table(
f""" 
SELECT 
  t1.year,
  t1.quarter,
  t1.case_number,
  t1.receipt_date,
  t1.event,
  t1.event_model,
  t1.order_type,
  t1.order_code,
  t1.order_desc,
  t1.child_role_id,
  t1.order_case_type,
  t2.case_DOI,
  CASE WHEN  t2.case_DOI is null
        THEN t1.receipt_date
       WHEN t1.event_model = 'U22' AND (t2.Case_DOI<t1.Receipt_date)
        THEN t2.Case_DOI
        ELSE t1.Receipt_date END
      AS Start_date
FROM 
  __temp__.ca_apps_child_d AS t1
      LEFT JOIN __temp__.DOI AS t2 
       ON t1.case_number = t2.case_number

""",

"child_doi")

In [ ]:
pydb.create_temp_table(
f""" 
WITH app_rank AS (

SELECT 
  *,
  RANK() OVER(PARTITION BY case_number, child_role_id
                       ORDER BY start_date) 
      AS app_rank
FROM 
  __temp__.child_doi
)

SELECT
  *
FROM
  app_rank
WHERE
  app_rank = 1

""",

"earliest_app")

In [ ]:
pydb.create_temp_table(
f""" 
SELECT 
  a.*,
  d.disp_date,
  date_diff('day',a.start_date, d.disp_date) AS days,
  date_diff('week',a.start_date, d.disp_date) AS weeks,
  ROW_NUMBER() OVER(PARTITION BY a.case_number, a.child_role_id, a.order_code
                       ORDER BY d.disp_date) 
      AS disp_rank   
FROM
  __temp__.earliest_app a
  LEFT JOIN __temp__.ca_disps_all_children_d d
    ON a.case_number = d.case_number
    AND a.child_role_id = d.child_role_id
WHERE 
  date_diff('day',a.start_date, d.disp_date) >= 0

""",

"app_disp_match")

In [ ]:
pydb.read_sql_query ("select * from __temp__.app_disp_match where case_number = 'PE17P00021' and child_role_id = 7630003 limit 10")

In [ ]:
pydb.read_sql_query ("select * from __temp__.ca_disps_all_children_d where case_number = 'PE17P00021' and child_role_id = 7630003")